# Poker Analysis

In [2]:
ranks = [str(n) for n in range(2, 11)] + list('JQKA')
suits = '♠️♥️♦️♣️'
suits = [s for s in suits][::2]

In [3]:
cards = []

for suit in suits:
    
    for rank in ranks:
        
        cards.append(suit+rank)
        
print(cards, '\n')

# check card count
print(len(cards))

['♠2', '♠3', '♠4', '♠5', '♠6', '♠7', '♠8', '♠9', '♠10', '♠J', '♠Q', '♠K', '♠A', '♥2', '♥3', '♥4', '♥5', '♥6', '♥7', '♥8', '♥9', '♥10', '♥J', '♥Q', '♥K', '♥A', '♦2', '♦3', '♦4', '♦5', '♦6', '♦7', '♦8', '♦9', '♦10', '♦J', '♦Q', '♦K', '♦A', '♣2', '♣3', '♣4', '♣5', '♣6', '♣7', '♣8', '♣9', '♣10', '♣J', '♣Q', '♣K', '♣A'] 

52


In [4]:
from itertools import combinations

hands = list(combinations(cards, 5))
print(hands[:10])

[('♠2', '♠3', '♠4', '♠5', '♠6'), ('♠2', '♠3', '♠4', '♠5', '♠7'), ('♠2', '♠3', '♠4', '♠5', '♠8'), ('♠2', '♠3', '♠4', '♠5', '♠9'), ('♠2', '♠3', '♠4', '♠5', '♠10'), ('♠2', '♠3', '♠4', '♠5', '♠J'), ('♠2', '♠3', '♠4', '♠5', '♠Q'), ('♠2', '♠3', '♠4', '♠5', '♠K'), ('♠2', '♠3', '♠4', '♠5', '♠A'), ('♠2', '♠3', '♠4', '♠5', '♥2')]


In [8]:
# create product function
def prod(nums):
    
    res = 1
    for n in nums:
        res *= n
    
    return res

# use product function to create factorial function
def fac(n):
    
    return prod(range(1, n+1))


# use factorial function to create combination function
def combin(n, k):
    
    return int(fac(n) / (fac(n-k) * fac(k)))

# check combinations
print(len(hands))
print(combin(52, 5))

2598960
2598960
